In [253]:
from SpacetimeVar import *

In [254]:
dim = 3
path = "../Source/Z4c/"

In [255]:
fileRHS = open(path+"ET_Integration_Rhs_K.H", "w+")
fileRHS.write("#ifndef ET_INTEGRATION_RHS_K_H \n")
fileRHS.write("#define ET_INTEGRATION_RHS_K_H \n\n")

fileRHS.write("#include <AMReX_REAL.H> \n")
fileRHS.write("#include <AMReX_Array4.H> \n")
fileRHS.write("#include <ET_Integration_Setup.H> \n\n")

fileRHS.write("AMREX_GPU_DEVICE \ninline \nvoid \n")
fileRHS.write("state_rhs(int i, int j, int k, \n")
fileRHS.write("        amrex::Array4<amrex::Real> const& rhs_fab, \n")
fileRHS.write("        amrex::Array4<amrex::Real const> const& state_fab, \n")
fileRHS.write("        amrex::GpuArray<amrex::Real,AMREX_SPACEDIM> const& dx, \n")
fileRHS.write("        const amrex::GeometryData& geom) noexcept \n{\n")
fileRHS.write("        const auto domain_xlo = geom.ProbLo(); \n\n")
fileRHS.write("        amrex::Real x0 = (i + 0.5)*geom.CellSize(0) + domain_xlo[0]; \n")
fileRHS.write("        amrex::Real x1 = (j + 0.5)*geom.CellSize(0) + domain_xlo[1]; \n")
fileRHS.write("        amrex::Real x2 = (k + 0.5)*geom.CellSize(0) + domain_xlo[2]; \n\n")

71

In [256]:
stVar.decl = []

phi = stVar('phi', declare = True)
gamtildeLL = stVarRank2('gamtildeLL', sym = 'sym01', declare = True)
Khat = stVar('Khat', declare = True)
AtildeLL = stVarRank2('AtildeLL', sym = 'sym01', declare = True)
theta = stVar('theta', declare = True)
GamtildeU = stVarRank1('GamtildeU', declare = True)

In [257]:
alpha = stVar('alpha', declare = True)
betaU = stVarRank1('betaU', declare = True)
BU = stVarRank1('BU', declare = True)

In [258]:
chi = stVar('chi')
chi.var = sp.exp(-4*phi.symb)

In [259]:
fileRHS.write(phi.AMReXDeclare())
fileRHS.write(gamtildeLL.AMReXDeclare())
fileRHS.write(Khat.AMReXDeclare())
fileRHS.write(AtildeLL.AMReXDeclare())
fileRHS.write(theta.AMReXDeclare())
fileRHS.write(GamtildeU.AMReXDeclare())

fileRHS.write(alpha.AMReXDeclare())
fileRHS.write(betaU.AMReXDeclare())
fileRHS.write(BU.AMReXDeclare())

fileRHS.write(chi.AMReXReal())


44

In [260]:
betaUp = stVarRank1('betaUp')
betaDn = stVarRank1('betaDn')

In [261]:
betaString = """
        amrex::Real betaUpOp0 = 0;
        amrex::Real betaDnOp0 = 0;
        if(betaU0 > 0)
        {
            betaUpOp0 = 1;
            betaDnOp0 = 0;
        }
        else if(betaU0 < 0)
        {
            betaUpOp0 = 0;
            betaDnOp0 = 1;
        }
        
        amrex::Real betaUpOp1 = 0;
        amrex::Real betaDnOp1 = 0;
        if(betaU1 > 0)
        {
            betaUpOp1 = 1;
            betaDnOp1 = 0;
        }
        else if(betaU1 < 0)
        {
            betaUpOp1 = 0;
            betaDnOp1 = 1;
        }
        
        amrex::Real betaUpOp2 = 0;
        amrex::Real betaDnOp2 = 0;
        if(betaU2 > 0)
        {
            betaUpOp2 = 1;
            betaDnOp2 = 0;
        }
        else if(betaU2 < 0)
        {
            betaUpOp2 = 0;
            betaDnOp2 = 1;
        }


"""
fileRHS.write(betaString)

828

In [262]:
DiffOrderUpDn = 2

In [263]:
betaDphi = stVar('betaDphi')
for k in range(3):
    betaDphi.var += betaU.symb[k]*(betaUp.var[k]*Dvar(phi.var,str(k),'up',DiffOrderUpDn)+betaDn.var[k]*Dvar(phi.var,str(k),'dn',DiffOrderUpDn))

betaDgamtildeLL = stVarRank2('betaDgamtildeLL')
for i in range(3):
    for j in range(3):
        for k in range(3):
            betaDgamtildeLL.var[i][j] += betaU.symb[k]*(betaUp.var[k]*Dvar(gamtildeLL.var[i][j],str(k),'up',DiffOrderUpDn)+betaDn.var[k]*Dvar(gamtildeLL.var[i][j],str(k),'dn',DiffOrderUpDn))

betaDKhat = stVar('betaDKhat')
for k in range(3):
    betaDKhat.var += betaU.symb[k]*(betaUp.var[k]*Dvar(Khat.var,str(k),'up',DiffOrderUpDn)+betaDn.var[k]*Dvar(Khat.var,str(k),'dn',DiffOrderUpDn))

betaDAtildeLL = stVarRank2('betaDAtildeLL')
for i in range(3):
    for j in range(3):
        for k in range(3):
            betaDAtildeLL.var[i][j] += betaU.symb[k]*(betaUp.var[k]*Dvar(AtildeLL.var[i][j],str(k),'up',DiffOrderUpDn)+betaDn.var[k]*Dvar(AtildeLL.var[i][j],str(k),'dn',DiffOrderUpDn))

betaDtheta = stVar('betaDtheta')
for k in range(3):
    betaDtheta.var += betaU.symb[k]*(betaUp.var[k]*Dvar(theta.var,str(k),'up',DiffOrderUpDn)+betaDn.var[k]*Dvar(theta.var,str(k),'dn',DiffOrderUpDn))

betaDGamtildeU = stVarRank1('betaDGamtildeU')
for i in range(3):
    for k in range(3):
        betaDGamtildeU.var[i] += betaU.symb[k]*(betaUp.var[k]*Dvar(GamtildeU.var[i],str(k),'up',DiffOrderUpDn)+betaDn.var[k]*Dvar(GamtildeU.var[i],str(k),'dn',DiffOrderUpDn))

betaDalpha = stVar('betaDalpha')
for k in range(3):
    betaDalpha.var += betaU.symb[k]*(betaUp.var[k]*Dvar(alpha.var,str(k),'up',DiffOrderUpDn)+betaDn.var[k]*Dvar(alpha.var,str(k),'dn',DiffOrderUpDn))
    
betaDbetaU = stVarRank1('betaDbetaU')
for i in range(3):
    for k in range(3):
        betaDbetaU.var[i] += betaU.symb[k]*(betaUp.var[k]*Dvar(betaU.var[i],str(k),'up',DiffOrderUpDn)+betaDn.var[k]*Dvar(betaU.var[i],str(k),'dn',DiffOrderUpDn))

betaDBU = stVarRank1('betaDBU')
for i in range(3):
    for k in range(3):
        betaDBU.var[i] += betaU.symb[k]*(betaUp.var[k]*Dvar(BU.var[i],str(k),'up',DiffOrderUpDn)+betaDn.var[k]*Dvar(BU.var[i],str(k),'dn',DiffOrderUpDn))



In [264]:
fileRHS.write(betaDphi.AMReXReal())
fileRHS.write(betaDgamtildeLL.AMReXReal())
fileRHS.write(betaDKhat.AMReXReal())
fileRHS.write(betaDAtildeLL.AMReXReal())
fileRHS.write(betaDtheta.AMReXReal())
fileRHS.write(betaDGamtildeU.AMReXReal())

fileRHS.write(betaDalpha.AMReXReal())
fileRHS.write(betaDbetaU.AMReXReal())
fileRHS.write(betaDBU.AMReXReal())


102

In [265]:
DiffOrder = 2
KOsigma = 0.15

In [266]:
fileRHS.write(DstVar(phi, 1, orderD = DiffOrder).AMReXDeclare())
fileRHS.write(DstVar(phi, 2, orderD = DiffOrder).AMReXDeclare())
fileRHS.write(DstVar(phi, 'KO', orderD = DiffOrder, sigma = KOsigma).AMReXDeclare())

fileRHS.write(DstVarRank2(gamtildeLL,1,orderD = DiffOrder).AMReXDeclare())
fileRHS.write(DstVarRank2(gamtildeLL,2,orderD = DiffOrder).AMReXDeclare())
fileRHS.write(DstVarRank2(gamtildeLL, 'KO', sigma = KOsigma).AMReXDeclare())

fileRHS.write(DstVar(Khat,1,orderD = DiffOrder).AMReXDeclare())
fileRHS.write(DstVar(Khat, 'KO', sigma = KOsigma,orderD = DiffOrder).AMReXDeclare())

fileRHS.write(DstVarRank2(AtildeLL,1, orderD = DiffOrder).AMReXDeclare())
fileRHS.write(DstVarRank2(AtildeLL, 'KO', sigma = KOsigma, orderD = DiffOrder).AMReXDeclare())

fileRHS.write(DstVar(theta,1, orderD = DiffOrder).AMReXDeclare())
fileRHS.write(DstVar(theta, 'KO', sigma = KOsigma, orderD = DiffOrder).AMReXDeclare())

fileRHS.write(DstVarRank1(GamtildeU, 1, orderD = DiffOrder).AMReXDeclare())
fileRHS.write(DstVarRank1(GamtildeU, 'KO', sigma = KOsigma, orderD = DiffOrder).AMReXDeclare())

fileRHS.write(DstVar(alpha, 1, orderD = DiffOrder).AMReXDeclare())
fileRHS.write(DstVar(alpha, 2, orderD = DiffOrder).AMReXDeclare())
fileRHS.write(DstVar(alpha, 'KO', sigma = KOsigma, orderD = DiffOrder).AMReXDeclare())

fileRHS.write(DstVarRank1(betaU, 1, orderD = DiffOrder).AMReXDeclare())
fileRHS.write(DstVarRank1(betaU, 2, orderD = DiffOrder).AMReXDeclare())
fileRHS.write(DstVarRank1(betaU, 'KO', sigma = KOsigma, orderD = DiffOrder).AMReXDeclare())

fileRHS.write(DstVarRank1(BU, 1, orderD = DiffOrder).AMReXDeclare())
fileRHS.write(DstVarRank1(BU, 'KO', sigma = KOsigma, orderD = DiffOrder).AMReXDeclare())


1956

In [267]:
gamtildeUU = stVarRank2('gamtildeUU')
for i in range(dim):
    for j in range(dim):
         gamtildeUU.var[i][j] += sp.simplify(sp.Matrix(gamtildeLL.symb).inv()[i,j])

In [268]:
detgamtilde = stVar('detgamtilde')
detgamtilde.var = sp.simplify(sp.Matrix(gamtildeLL.symb).det())

In [269]:
print(detgamtilde.var)

gamtildeLL00*gamtildeLL11*gamtildeLL22 - gamtildeLL00*gamtildeLL12*gamtildeLL21 - gamtildeLL01*gamtildeLL10*gamtildeLL22 + gamtildeLL01*gamtildeLL12*gamtildeLL20 + gamtildeLL02*gamtildeLL10*gamtildeLL21 - gamtildeLL02*gamtildeLL11*gamtildeLL20


In [270]:
fileRHS.write(gamtildeUU.AMReXReal())

3026

In [271]:
DgamtildeLLL = stVarRank3('DgambarLLL')
GamtildeULL = stVarRank3('GamtildeULL')
GamtildeLLL = stVarRank3('GamtildeLLL')
GamULL = stVarRank3('GamULL')

for i in range(dim):
    for j in range(dim):
        for k in range(dim):
            DgamtildeLLL.var[i][j][k] = Dsymb(gamtildeLL.symb[i][j],k)
            
for i in range(dim):
    for j in range(dim):
        for k in range(dim):
            for l in range(dim):
                GamtildeULL.var[i][j][k] += 1/2*gamtildeUU.symb[i][l]*(DgamtildeLLL.var[l][j][k]+DgamtildeLLL.var[l][k][j]-DgamtildeLLL.var[j][k][l])
                
for i in range(dim):
    for j in range(dim):
        for k in range(dim):
            for l in range(dim):
                GamtildeLLL.var[i][j][k] += gamtildeLL.symb[i][l]*GamtildeULL.var[l][j][k]
                
for i in range(dim):
    for j in range(dim):
        for k in range(dim):
            GamULL.var[i][j][k] += GamtildeULL.symb[i][j][k] + 2*(sp.eye(3)[i,j]*Dsymb(phi.symb,k)+sp.eye(3)[i,k]*Dsymb(phi.symb,j))
            for l in range(3):
                GamULL.var[i][j][k] += -2*gamtildeLL.symb[j][k]*gamtildeUU.symb[i][l]*Dsymb(phi.symb,l)
                               
                

In [272]:
fileRHS.write(GamtildeULL.AMReXReal())
fileRHS.write(GamtildeLLL.AMReXReal())
fileRHS.write(GamULL.AMReXReal())

4362

In [273]:
DDalphaLL = stVarRank2('DDalphaLL')
for i in range(3):
    for j in range(3):
        DDalphaLL.var[i][j] += Dsymb(alpha.symb,str(i)+str(j))
        for k in range(3):
            DDalphaLL.var[i][j] += -GamULL.symb[k][i][j]*Dsymb(alpha.symb,str(k))

In [274]:
fileRHS.write(DDalphaLL.AMReXReal())

954

In [275]:
gamLL = stVarRank2('gamLL')
for i in range(3):
    for j in range(3):
        gamLL.var[i][j] = sp.exp(4*phi.symb)*gamtildeLL.symb[i][j]
        
gamUU = stVarRank2('gamUU')
for i in range(3):
    for j in range(3):
        gamUU.var[i][j] = sp.exp(-4*phi.symb)*gamtildeUU.symb[i][j]

In [276]:
fileRHS.write(gamLL.AMReXReal())
fileRHS.write(gamUU.AMReXReal())

549

In [277]:
AtildeUU = stVarRank2('AtildeUU')
for i in range(3):
    for j in range(3):
        for k in range(3):
            for l in range(3):
                AtildeUU.var[i][j] += gamtildeUU.symb[i][k]*gamtildeUU.symb[j][l]*AtildeLL.symb[k][l]
                
AtildeUL = stVarRank2('AtildeUL')
for i in range(3):
    for j in range(3):
        for k in range(3):
            AtildeUL.var[i][j] += gamtildeUU.symb[i][k]*AtildeLL.symb[k][j]

In [278]:
TrAtilde = stVar('TrAtilde')
for i in range(3):
    for j in range(3):
        TrAtilde.var += gamtildeUU.symb[i][j]*AtildeLL.symb[i][j]

In [279]:
print(TrAtilde.AMReXReal())

        amrex::Real TrAtilde = 0 + AtildeLL00*gamtildeUU00 + AtildeLL01*gamtildeUU01 + AtildeLL02*gamtildeUU02 + AtildeLL10*gamtildeUU10 + AtildeLL11*gamtildeUU11 + AtildeLL12*gamtildeUU12 + AtildeLL20*gamtildeUU20 + AtildeLL21*gamtildeUU21 + AtildeLL22*gamtildeUU22;



In [280]:
fileRHS.write(AtildeUU.AMReXReal())
fileRHS.write(AtildeUL.AMReXReal())

990

In [281]:
DDalphaTFLL=stVarRank2('DDalphaTFLL')

for i in range(3):
    for j in range(3):
        DDalphaTFLL.var[i][j] += DDalphaLL.symb[i][j]
        for k in range(3):
            for l in range(3):
                DDalphaTFLL.var[i][j] += -1/3*gamLL.symb[i][j]*(gamUU.symb[k][l]*DDalphaLL.symb[k][l])

In [282]:
fileRHS.write(DDalphaTFLL.AMReXReal())

4500

In [283]:
GamtildeDU = stVarRank1('GamtildeDU')
for i in range(dim):
    for j in range(dim):
        for k in range(dim):
            GamtildeDU.var[i] += GamtildeULL.symb[i][j][k]*gamtildeUU.symb[j][k] 

In [284]:
fileRHS.write(GamtildeDU.AMReXReal())

909

In [285]:
RtildeLL = stVarRank2('RtildeLL')

for i in range(3):
    for j in range(3):
        for l in range(3):
            for m in range(3):
                RtildeLL.var[i][j] += -1/2*gamtildeUU.symb[l][m]*Dsymb(gamtildeLL.symb[i][j],str(l)+str(m))
        for k in range(3):
            RtildeLL.var[i][j] += 1/2*(gamtildeLL.symb[k][i]*Dsymb(GamtildeU.symb[k],str(j))+gamtildeLL.symb[k][j]*Dsymb(GamtildeU.symb[k],str(i)))
            RtildeLL.var[i][j] += 1/2*(GamtildeDU.symb[k]*GamtildeLLL.symb[i][j][k]+GamtildeDU.symb[k]*GamtildeLLL.symb[j][i][k])
            #RtildeLL.var[i][j] += 1/2*GamtildeU.symb[k]*(symb_GambarLLL[i][j][k]+symb_GambarLLL[j][i][k])
            #RtildeLL.var[i][j] += 1/2*(symb_gambarLL[k][i]*der(symb_GambarU[k],j)+symb_gambarLL[k][j]*der(symb_GambarU[k],i))
            
for i in range(3):
    for j in range(3):
        for k in range(3):
            for l in range(3):
                for m in range(3):
                    RtildeLL.var[i][j] += gamtildeUU.symb[l][m]*(GamtildeULL.symb[k][l][i]*GamtildeLLL.symb[j][k][m]+GamtildeULL.symb[k][l][j]*GamtildeLLL.symb[i][k][m])
                    RtildeLL.var[i][j] += gamtildeUU.symb[l][m]*GamtildeULL.symb[k][i][m]*GamtildeLLL.symb[k][l][j]

In [286]:
fileRHS.write(RtildeLL.AMReXReal())

33534

In [287]:
DtildeDtildephiLL = stVarRank2('DtildeDtildephiLL')
for i in range(3):
    for j in range(3):
        DtildeDtildephiLL.var[i][j] += Dsymb(phi.symb,str(i)+str(j))
        for k in range(3):
            DtildeDtildephiLL.var[i][j] += -GamtildeULL.symb[k][i][j]*Dsymb(phi.symb,k)


In [288]:
fileRHS.write(DtildeDtildephiLL.AMReXReal())

1089

In [289]:
RphiLL = stVarRank2('RphiLL')            
for i in range(3):
    for j in range(3):
        RphiLL.var[i][j] += -2*(DtildeDtildephiLL.symb[i][j])+4*Dsymb(phi.symb,str(i))*Dsymb(phi.symb,str(j))
        for l in range(3):
            for m in range(3):
                RphiLL.var[i][j] += -2*gamtildeLL.symb[i][j]*gamtildeUU.symb[l][m]*DtildeDtildephiLL.symb[l][m]
                RphiLL.var[i][j] += -4*gamtildeLL.symb[i][j]*gamtildeUU.symb[l][m]*Dsymb(phi.symb,str(l))*Dsymb(phi.symb,str(m))                

In [290]:
fileRHS.write(RphiLL.AMReXReal())

8301

In [291]:
RLL = stVarRank2('RLL')
for i in range(3):
    for j in range(3):
        RLL.var[i][j] += RtildeLL.symb[i][j]+RphiLL.symb[i][j]

In [292]:
fileRHS.write(RLL.AMReXReal())

459

In [293]:
Rsclr = stVar('Rsclr')
for i in range(3):
    for j in range(3):
        Rsclr.var += gamUU.symb[i][j]*RLL.symb[i][j] 

In [294]:
fileRHS.write(Rsclr.AMReXReal())

175

In [295]:
RTFLL = stVarRank2('RTFLL')
for i in range(3):
    for j in range(3):
        RTFLL.var[i][j] = RLL.symb[i][j]
        for l in range(3):
            for m in range(l,3):
                RTFLL.var[i][j] += -1/3*gamLL.symb[i][j]*gamUU.symb[l][m]*RLL.symb[l][m]

In [296]:
fileRHS.write(RTFLL.AMReXReal())

2718

In [297]:
RHS_phi = stVar('phi')
RHS_phi.var = -1/6*alpha.symb*(Khat.symb+2*theta.symb)
RHS_phi.var += betaDphi.symb 
for k in range(3):
    RHS_phi.var += 1/6*Dsymb(betaU.symb[k],str(k))#+betaU.symb[k]*Dsymb(phi.symb,str(k))
    
RHS_phi.var += Dsymb(phi.symb,'KO')

In [298]:
fileRHS.write(RHS_phi.AMReXRHS())

194

In [299]:
RHS_gamtildeLL = stVarRank2('gamtildeLL',sym = 'sym01')

for i in range(3):
    for j in range(3):
        RHS_gamtildeLL.var[i][j] += -2*alpha.symb*AtildeLL.symb[i][j]
        RHS_gamtildeLL.var[i][j] += betaDgamtildeLL.symb[i][j]
        for k in range(3):
            RHS_gamtildeLL.var[i][j] += gamtildeLL.symb[i][k]*Dsymb(betaU.symb[k],str(j))+gamtildeLL.symb[j][k]*Dsymb(betaU.symb[k],str(i))-2/3*gamtildeLL.symb[i][j]*Dsymb(betaU.symb[k],str(k))#+betaU.symb[k]*Dsymb(gamtildeLL.symb[i][j],str(k))
            
for i in range(3):
    for j in range(3):
        RHS_gamtildeLL.var[i][j] +=Dsymb(gamtildeLL.symb[i][j],'KO')
   

In [300]:
fileRHS.write(RHS_gamtildeLL.AMReXRHS())

1935

In [301]:
kappa1 = stVar('kappa1')
kappa2 = stVar('kappa2')

kappa1.var = 0.1
kappa2.var = 0

RHS_Khat = stVar('Khat')
for i in range(3):
    for j in range(3):
        RHS_Khat.var += -gamUU.symb[i][j]*DDalphaLL.symb[j][i]+alpha.symb*AtildeLL.symb[i][j]*AtildeUU.symb[i][j]

RHS_Khat.var += betaDKhat.symb
#for k in range(3):
 #   RHS_Khat.var += betaU.symb[k]*Dsymb(Khat.symb,str(k))

RHS_Khat.var += 1/3*alpha.symb*((Khat.symb+2*theta.symb)**2+kappa1.symb*(1-kappa2.symb)*theta.symb)
RHS_Khat.var += Dsymb(Khat.symb,'KO')


In [302]:
fileRHS.write(kappa1.AMReXReal())
fileRHS.write(kappa2.AMReXReal())
fileRHS.write(RHS_Khat.AMReXRHS())

617

In [303]:
RHS_AtildeLL = stVarRank2('AtildeLL', sym = 'sym01')
for i in range(3):
    for j in range(3):
        RHS_AtildeLL.var[i][j] += chi.symb*(-DDalphaTFLL.symb[i][j]+alpha.symb*RTFLL.symb[i][j])+alpha.symb*(Khat.symb+2*theta.symb)*AtildeLL.symb[i][j]
        RHS_AtildeLL.var[i][j] += betaDAtildeLL.symb[i][j]
        for k in range(3):
            RHS_AtildeLL.var[i][j] += -2*alpha.symb*AtildeLL.symb[i][k]*AtildeUL.symb[k][j]#+betaU.symb[k]*Dsymb(AtildeLL.symb[i][j],str(k))
            RHS_AtildeLL.var[i][j] += AtildeLL.symb[i][k]*Dsymb(betaU.symb[k],str(j))+AtildeLL.symb[j][k]*Dsymb(betaU.symb[k],str(i))-2/3*AtildeLL.symb[i][j]*Dsymb(betaU.symb[k],str(k))
            
for i in range(3):
    for j in range(3):
        RHS_AtildeLL.var[i][j] += Dsymb(AtildeLL.symb[i][j],'KO')

In [304]:
fileRHS.write(RHS_AtildeLL.AMReXRHS())

2721

In [305]:
RHS_theta = stVar('theta')
RHS_theta.var = 1/2*alpha.symb*(Rsclr.symb + 2/3*(Khat.symb+2*theta.symb)**2 - 2*kappa1.symb*(2+kappa2.symb)*theta.symb)
#for i in range(3):
 #   RHS_theta.var += betaU.symb[i]*Dsymb(theta.symb,str(i))

RHS_theta.var += betaDtheta.symb
for i in range(3):
    for j in range(3):
        RHS_theta.var += -1/2*alpha.symb*AtildeLL.symb[i][j]*AtildeUU.symb[i][j]

RHS_theta.var += Dsymb(theta.symb,'KO')

In [306]:
fileRHS.write(RHS_theta.AMReXRHS())

469

In [307]:
RHS_GamtildeU = stVarRank1('GamtildeU')

for i in range(3):
    RHS_GamtildeU.var[i] += betaDGamtildeU.symb[i]
    for j in range(3):
        RHS_GamtildeU.var[i] += -2*AtildeUU.symb[i][j]*Dsymb(alpha.symb,str(j))+2*alpha.symb*(-1/3*gamtildeUU.symb[i][j]*(2*Dsymb(Khat.symb,str(j))+Dsymb(theta.symb,str(j)))+6*AtildeUU.symb[i][j]*Dsymb(phi.symb,str(j)))
        RHS_GamtildeU.var[i] += -GamtildeDU.symb[j]*Dsymb(betaU.symb[i],str(j))+2/3*GamtildeDU.symb[i]*Dsymb(betaU.symb[j],str(j))#+betaU.symb[j]*Dsymb(GamtildeU.symb[i],str(j))
        for k in range(3):
            RHS_GamtildeU.var[i] += 2*alpha.symb*GamtildeULL.symb[i][j][k]*AtildeUU.symb[k][j]
            RHS_GamtildeU.var[i] += 1/3*gamtildeUU.symb[k][i]*Dsymb(betaU.symb[j],str(j)+str(k))+gamtildeUU.symb[j][k]*Dsymb(betaU.symb[i],str(j)+str(k))
            
for i in range(3):
    RHS_GamtildeU.var[i] += -2*alpha.symb*kappa1.symb*(GamtildeU.symb[i]-GamtildeDU.symb[i])
for i in range(3):
    RHS_GamtildeU.var[i] += Dsymb(GamtildeU.symb[i],'KO')

In [308]:
fileRHS.write(RHS_GamtildeU.AMReXRHS())

4716

In [309]:
RHS_alpha = stVar('alpha')

RHS_alpha.var = -2*alpha.symb*Khat.symb+betaDalpha.symb
#for i in range(3):
 #   RHS_alpha.var += betaU.symb[i]*Dsymb(alpha.symb,str(i))
RHS_alpha.var += Dsymb(alpha.symb,'KO')

In [310]:
fileRHS.write(RHS_alpha.AMReXRHS())

77

In [311]:
RHS_betaU = stVarRank1('betaU')
for i in range(3):
    RHS_betaU.var[i] += 3/4*BU.symb[i]+betaDbetaU.symb[i]
    #for j in range(3):
     #   RHS_betaU.var[i] += betaU.symb[j]*Dsymb(betaU.symb[i],str(j))
        
for i in range(3):
    RHS_betaU.var[i] += Dsymb(betaU.symb[i],'KO')

In [312]:
fileRHS.write(RHS_betaU.AMReXRHS())

228

In [313]:
RHS_BU = stVarRank1('BU')
eta = stVar('eta')
eta.var = 2
for i in range(3):
    RHS_BU.var[i] += RHS_GamtildeU.var[i]-eta.symb*BU.symb[i]+betaDBU.symb[i]
    #for j in range(3):
     #   RHS_BU.var[i] += betaU.symb[j]*Dsymb(BU.symb[i],j)
        
for i in range(3):
    for j in range(3):
        RHS_BU.var[i] += -betaU.symb[j]*Dsymb(GamtildeU.symb[i],j)
        
for i in range(3):
    RHS_BU.var[i] += Dsymb(BU.symb[i],'KO')

In [314]:
fileRHS.write(eta.AMReXReal())

fileRHS.write(RHS_BU.AMReXRHS())

4980

In [315]:
fileRHS.write("}\n")
fileRHS.write("#endif")
fileRHS.close()

In [59]:
fileINIT = open(path+"ET_Integration_Init_K.H", "w+")
fileINIT.write("#ifndef ET_INTEGRATION_INIT_K_H \n")
fileINIT.write("#define ET_INTEGRATION_INIT_K_H \n\n")

fileINIT.write("#include <AMReX_REAL.H> \n")
fileINIT.write("#include <AMReX_Array4.H> \n")
fileINIT.write("#include <ET_Integration_Setup.H> \n\n")

fileINIT.write("AMREX_GPU_DEVICE \ninline \nvoid \n")
fileINIT.write("state_init(int i, int j, int k, \n")
fileINIT.write("        amrex::Array4<amrex::Real> const& state_fab, \n")
fileINIT.write("        amrex::Real time, const amrex::GeometryData& geom) noexcept \n{\n")
fileINIT.write("        const auto domain_xlo = geom.ProbLo(); \n\n")
fileINIT.write("        amrex::Real x0 = (i + 0.5)*geom.CellSize(0) + domain_xlo[0];\n")
fileINIT.write("        amrex::Real x1 = (j + 0.5)*geom.CellSize(1) + domain_xlo[1];\n")
fileINIT.write("        amrex::Real x2 = (k + 0.5)*geom.CellSize(2) + domain_xlo[2];\n\n")

70

In [60]:
x = stVar('x0')
y = stVar('x1')
z = stVar('x2')
r = stVar('r')
r.var = sp.sqrt(x.symb**2+y.symb**2+z.symb**2)

c1 = stVar('c1')
c2 = stVar('c2')
c1.var = 3.0
c2.var = -3.0

r1 = stVar('r1')
r1.var = sp.sqrt((x.symb - c1.symb)**2 + y.symb**2 + z.symb**2)

r2 = stVar('r2')
r2.var = sp.sqrt((x.symb - c2.symb)**2 + y.symb**2 + z.symb**2)

M1 = stVar('M1')
M2 = stVar('M2')
M1.var = 2.0
M2.var = 2.0

In [61]:
fileINIT.write(r.AMReXReal())
fileINIT.write(c1.AMReXReal())
fileINIT.write(c2.AMReXReal())
fileINIT.write(r1.AMReXReal())
fileINIT.write(r2.AMReXReal())
fileINIT.write(M1.AMReXReal())
fileINIT.write(M2.AMReXReal())

30

In [62]:
Init_phi = stVar('phi')
Init_phi.var = sp.log(1+M1.symb/(2*r1.symb)+M2.symb/(2*r2.symb))

In [63]:
fileINIT.write(Init_phi.AMReXInit())

89

In [64]:
Init_gamtildeLL = stVarRank2('gamtildeLL', sym = 'sym01')
Init_gamtildeLL.var = np.array(sp.eye(3))        

In [65]:
fileINIT.write(Init_gamtildeLL.AMReXInit())

312

In [66]:
Init_Khat = stVar('Khat')
Init_Khat.var = 0

In [67]:
fileINIT.write(Init_Khat.AMReXInit())

44

In [68]:
Init_AtildeLL = stVarRank2('AtildeLL', sym = 'sym01')
Init_AtildeLL.var = np.array(sp.zeros(3))

In [69]:
fileINIT.write(Init_AtildeLL.AMReXInit())

300

In [70]:
Init_theta = stVar('theta')
Init_theta.var = 0

In [71]:
fileINIT.write(Init_theta.AMReXInit())

45

In [72]:
Init_GamtildeU = stVarRank1('GamtildeU')
Init_GamtildeU.var = np.array([0, 0, 0])

In [73]:
fileINIT.write(Init_GamtildeU.AMReXInit())

150

In [74]:
Init_alpha = stVar('alpha')
Init_alpha.var = sp.exp(-2*Init_phi.var)

In [75]:
fileINIT.write(Init_alpha.AMReXInit())

95

In [76]:
Init_betaU = stVarRank1('betaU')
Init_betaU.var = np.array([0, 0, 0])

In [77]:
fileINIT.write(Init_betaU.AMReXInit())

138

In [78]:
Init_BU = stVarRank1('BU')
Init_BU.var = np.array([0, 0, 0])

In [79]:
fileINIT.write(Init_BU.AMReXInit())

129

In [80]:
fileINIT.write("}\n")
fileINIT.write("#endif")
fileINIT.close()

In [81]:
fileDiag = open(path+"ET_Integration_Constraints_K.H", "w+")
fileDiag.write("#ifndef ET_INTEGRATION_CONSTRAINTS_K_H \n")
fileDiag.write("#define ET_INTEGRATION_CONSTRAINTS_K_H \n\n")

fileDiag.write("#include <AMReX_REAL.H> \n")
fileDiag.write("#include <AMReX_Array4.H> \n")
fileDiag.write("#include <ET_Integration_Setup.H> \n\n")

fileDiag.write("AMREX_GPU_DEVICE \ninline \nvoid \n")
fileDiag.write("state_diagnostics(int i, int j, int k, \n")
fileDiag.write("        amrex::Array4<amrex::Real> const& diag, \n")
fileDiag.write("        amrex::Array4<amrex::Real const> const& state_fab, \n")
fileDiag.write("        amrex::GpuArray<amrex::Real,AMREX_SPACEDIM> const& dx, \n")
fileDiag.write("        const amrex::GeometryData& geom) noexcept \n{\n")
fileDiag.write("        const auto domain_xlo = geom.ProbLo(); \n\n")
fileDiag.write("        amrex::Real x0 = (i + 0.5)*geom.CellSize(0) + domain_xlo[0];\n")
fileDiag.write("        amrex::Real x1 = (j + 0.5)*geom.CellSize(1) + domain_xlo[1];\n")
fileDiag.write("        amrex::Real x2 = (k + 0.5)*geom.CellSize(2) + domain_xlo[2];\n\n")

#fileDiag.write("AMREX_GPU_DEVICE \ninline \nvoid \n")
#fileDiag.write("state_diagnostics(int i, int j, int k, \n")
#fileDiag.write("        amrex::Array4<amrex::Real> const& diag, \n")
#fileDiag.write("        amrex::Array4<amrex::Real const> const& state_fab, \n")
#fileDiag.write("        amrex::GpuArray<amrex::Real,AMREX_SPACEDIM> const& dx) noexcept \n{\n")

70

In [82]:
fileDiag.write(phi.AMReXDeclare())
fileDiag.write(gamtildeLL.AMReXDeclare())
fileDiag.write(Khat.AMReXDeclare())
fileDiag.write(AtildeLL.AMReXDeclare())
fileDiag.write(theta.AMReXDeclare())
fileDiag.write(GamtildeU.AMReXDeclare())

fileDiag.write(chi.AMReXReal())

44

In [83]:
fileDiag.write(DstVar(phi, 1, orderD = DiffOrder).AMReXDeclare())
fileDiag.write(DstVar(phi, 2, orderD = DiffOrder).AMReXDeclare())

fileDiag.write(DstVarRank2(gamtildeLL, 1, orderD = DiffOrder).AMReXDeclare())
fileDiag.write(DstVarRank2(gamtildeLL, 2, orderD = DiffOrder).AMReXDeclare())

fileDiag.write(DstVar(Khat, 1, orderD = DiffOrder).AMReXDeclare())

#fileDiag.write(DstVarRank2(AtildeLL,1).AMReXDeclare())

fileDiag.write(DstVar(theta, 1, orderD = DiffOrder).AMReXDeclare())

fileDiag.write(DstVarRank1(GamtildeU, 1, orderD = DiffOrder).AMReXDeclare())


1296

In [84]:
fileDiag.write(gamtildeUU.AMReXReal())

3026

In [85]:
fileDiag.write(AtildeUU.AMReXReal())

3447

In [86]:
fileDiag.write(GamtildeULL.AMReXReal())
fileDiag.write(GamtildeLLL.AMReXReal())

15552

In [87]:
fileDiag.write(GamtildeDU.AMReXReal())

909

In [88]:
fileDiag.write(RtildeLL.AMReXReal())

33534

In [89]:
Rtildesclr = stVar('Rtildesclr')
for i in range(3):
    for j in range(3):
        Rtildesclr.var += gamtildeUU.symb[i][j]*RtildeLL.symb[i][j] 

In [90]:
fileDiag.write(Rtildesclr.AMReXReal())

270

In [91]:
DtildeDtildephiLL = stVarRank2('DtildeDtildephiLL')

for i in range(3):
    for j in range(3):
        DtildeDtildephiLL.var[i][j] += Dsymb(phi.symb,str(i)+str(j))
        for k in range(3):
            DtildeDtildephiLL.var[i][j] += -GamtildeULL.symb[k][i][j]*Dsymb(phi.symb,str(k))

In [92]:
fileDiag.write(DtildeDtildephiLL.AMReXReal())

1089

In [93]:
ConstraintH = stVar('H', declareDiag = True)
for i in range(3):
    for j in range(3):
        ConstraintH.var += sp.exp(phi.symb)*gamtildeUU.symb[i][j]*(Dsymb(phi.symb,str(i)+str(j))+DtildeDtildephiLL.symb[i][j])+sp.exp(5*phi.symb)*AtildeLL.symb[i][j]*AtildeUU.symb[i][j]/8
ConstraintH.var += sp.exp(phi.symb)*Rtildesclr.symb/8-sp.exp(phi.symb)*((Khat.symb+2*theta.symb)**2)/12

In [94]:
fileDiag.write(ConstraintH.AMReXDiag())

1126

In [95]:
fileDiag.write("}\n")
fileDiag.write("#endif")
fileDiag.close()

In [96]:
nghostcells = 4

In [97]:
fileSETUP = open(path+"ET_Integration_Setup.H", "w+")
fileSETUP.write("#ifndef ET_INTEGRATION_SETUP_K_H \n")
fileSETUP.write("#define ET_INTEGRATION_SETUP_K_H \n\n")

fileSETUP.write("#include <AMReX_REAL.H> \n")
fileSETUP.write("#include <AMReX_Array4.H> \n\n")
    
fileSETUP.write("namespace Idx { \n")
fileSETUP.write("         enum ETIndexes {")
    
Idx_string = ""
for itr in stVar.decl:
    Idx_string += itr+", "
Idx_string += "NumScalars"
    
fileSETUP.write(Idx_string)
fileSETUP.write("}; \n};\n\n")
    
fileSETUP.write("namespace Diag { \n")
fileSETUP.write("         enum DiagnosticIndexes {")
    
Idx_string = ""
for itr in stVar.declDiag:
    Idx_string += itr+", "
Idx_string += "NumScalars"
    
fileSETUP.write(Idx_string)
fileSETUP.write("}; \n};\n\n")
    
fileSETUP.write("#define NUM_GHOST_CELLS "+str(nghostcells)+"\n\n")
fileSETUP.write("#endif")

fileSETUP.close()

In [98]:
fileVARS = open(path+"ET_Integration_Variables.H","w+")
fileVARS.write("names = {")
for itr in range(len(stVar.decl)-1):
    fileVARS.write("\""+stVar.decl[itr]+"\", ")
fileVARS.write("\""+stVar.decl[len(stVar.decl)-1]+"\"")
fileVARS.write("};")
fileVARS.close()